In [15]:
import numpy as np
#from moviepy.editor import VideoFileClip	import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Input, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

from moviepy.editor import VideoFileClip
import os

In [16]:
#the three following cells are used to recreate the model for which the weights were calculated
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB1(include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [17]:
top_layers = efficientnet_model.output
top_layers = Flatten(name="flatten_top")(top_layers)
top_layers = Dense(1024, activation="ReLU",name="first_dense_top")(top_layers)
top_layers = Dropout(0.5, name="dropout_top")(top_layers)
top_layers = Dense(units=3, activation="softmax",name="linear_output")(top_layers)

In [18]:
model = Model(inputs=efficientnet_model.input, outputs=top_layers)

In [19]:
#loading the weights
cp_dir = "training"
weights = tf.train.latest_checkpoint(cp_dir)
model.load_weights(weights)

In [20]:
video_file="79466_2019_V9.040EU_V9.039EU.mpg"
video_clip = VideoFileClip(video_file)

In [68]:
nb_frame_per_sec=1
framing_rate = min(video_clip.fps, nb_frame_per_sec)
step = 1 / video_clip.fps if framing_rate == 0 else 1 / framing_rate

In [69]:
#creating a new directory to store framed images
filename, _ = os.path.splitext(video_file)
filename += "-framed"
if not os.path.isdir(filename):
	os.mkdir(filename)

In [70]:
ts_Fissure = []
ts_Racines = []
for current_duration in np.arange(0, video_clip.duration, step):
    frame_filename = os.path.join(filename, f"frame{current_duration}.jpg")
    # save the frame with the current duration
    video_clip.save_frame(frame_filename, current_duration)
    img = tf.keras.utils.load_img(frame_filename, target_size=(224, 224))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = model.predict(img_array)
    
    pred = np.round(predictions[0])

    # need to know what labels [1, 0, 0], [0, 1, 0] and [0, 0, 1] correspond to
    # Assuming [1, 0, 0] is Fissure and [0, 0, 1] correspond to Racines
    if pred[0] == 1 and pred[1] == 0 and pred[2] == 0 :
        mins = np.floor(current_duration/60)
        secs = (current_duration/60 - mins)*60
        ts_Fissure.append([mins, secs])
        
    elif pred[0] == 0 and pred[1] == 0 and pred[2] == 1 :
        mins = np.floor(current_duration/60)
        secs = (current_duration/60 - mins)*60
        ts_Racines.append([mins, secs])


In [71]:
ts_Fissure=np.int_(np.floor(ts_Fissure))
ts_Racines=np.int_(np.floor(ts_Racines))

In [72]:

print('Fissure found in time stamps:')
"""for timeStamps in ts_Fissure:
    print('\n')
    print(timeStamps[0]+':'+timeStamps[1])"""

print('Racine found in time stamps:')
for timeStamps in ts_Racines:
    #print('\n')
    print(str(timeStamps[0])+'\''+str(timeStamps[1]) + '\"')

Fissure found in time stamps:
Racine found in time stamps:
0'0"
0'1"
0'2"
0'8"
0'9"
0'10"
0'11"
0'12"
0'13"
0'14"
0'15"
0'16"
0'19"
0'36"
0'37"
0'38"
0'39"
0'40"
0'41"
0'42"
0'43"
0'44"
0'45"
0'46"
0'47"
0'48"
0'49"
0'50"
0'51"
0'52"
1'6"
1'7"
1'7"
1'8"
1'10"
1'11"
1'11"
1'12"
1'14"
1'15"
1'15"
1'17"
1'18"
1'19"
1'19"
1'21"
1'22"
1'23"
1'23"
1'25"
1'26"
1'26"
1'27"
1'29"
1'38"
1'38"
1'40"
1'55"
1'56"
1'57"
1'57"
1'59"
2'0"
2'0"
2'1"
2'9"
2'22"
2'23"
2'23"
2'24"
2'25"
2'27"
2'28"
2'29"
2'30"
2'30"
2'35"
2'36"
2'37"
2'38"
2'38"
2'39"
2'40"
2'42"
2'43"
2'44"
2'45"
2'46"
2'46"
2'47"
2'49"
2'50"
2'51"
2'52"
2'53"
2'53"
2'54"
2'55"
3'5"
3'6"
3'20"
3'21"
3'24"
3'25"
3'27"
3'28"
3'32"
3'34"
3'35"
3'38"
3'38"
3'39"
3'40"
3'59"
4'11"
4'12"
4'13"
4'14"
4'15"
4'15"
4'16"
4'17"
4'18"
4'19"
4'20"
4'21"
4'23"
4'25"
4'26"
5'11"
5'30"
5'31"
5'33"
5'41"
